In [ ]:
!pip install google-cloud-aiplatform

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
import vertexai
import getpass
import requests
from vertexai.preview.language_models import TextGenerationModel

def predict_large_language_model_sample(
    project_id: str,
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    content: str,
    location: str = "us-central1",
    tuned_model_name: str = "",
    ) :
    """Predict using a Large Language Model."""
    vertexai.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained(model_name)
    if tuned_model_name:
      model = model.get_tuned_model(tuned_model_name)
    response = model.predict(
        content,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)
    print(f"Response from Model: {response.text}")
    return response.text

In [ ]:
api_key = 'xxxxxx'  # replace with your actual API key
company_list = ['Tesla', 'Apple', 'Google']

In [ ]:
def get_financial_news(company_name, api_key):
    url = f"https://api.goperigon.com/v1/all?source=cnn.com&q={company_name}&from=2023-05-10&sortBy=date&apiKey={api_key}"
    response = requests.get(url) 
    if response.status_code != 200:
        print(f"Request failed with status code {response.status_code}")
        return
    data = response.json()
    articles = data.get('articles')
    if articles is None:
        print("No articles found in the response.")
        return
    for article in articles[:5]:
        print(f"Title: {article.get('title')}")
        print(f"Description: {article.get('description')}")
        print(f"URL: {article.get('url')}")
        print("\n")
    return articles[:5]

In [ ]:
from dataclasses import dataclass
@dataclass
class CompanySentiment:
    url: str
    name: str
    sentiment: str

In [ ]:
company_sentiment_list = []
for company_name in company_list:
  print("The company name is", company_name)
  articles = get_financial_news(company_name, api_key)
  for article in articles:
    news_article = '''input: ''' + article.get('description') + '''Is the sentiment positive, negative or neutral of the given input? : '''
    print("Constructed news article is" , news_article)
    sentiment = predict_large_language_model_sample("keras_xxxx", "text-bison@001", 0.2, 5, 0.8, 1, news_article, "us-central1")
    print("sentiment of the ", article.get('url'), "is " , sentiment)
    company_sentiment_list.append(CompanySentiment(article.get('url'),company_name, sentiment))

The company name is Tesla
Title: Elon Musk claims George Soros 'hates humanity.' The ADL says Musk's attacks 'will embolden extremists'
Description: Elon Musk launched a baseless Twitter attack on George Soros, a frequent target for antisemitic conspiracy theories, accusing the financier of hating humanity days after Soros disclosed having sold off a modest stake in Tesla.
URL: https://www.cnn.com/2023/05/16/business/elon-musk-george-soros/index.html


Title: Elon Musk said he must approve all hiring decisions at Tesla
Description: Tesla Inc CEO Elon Musk has said that the company can make no new hires unless he personally approves them, according to a copy of the email seen by Reuters.
URL: https://www.cnn.com/2023/05/16/business/elon-musk-tesla-hiring-memo/index.html


Title: US Virgin Islands subpoenas Elon Musk as part of lawsuit into Jeffrey Epstein sex trafficking ring
Description: The US Virgin Islands has subpoenaed Elon Musk, requesting documents from the Tesla CEO and Twitter

In [ ]:
def print_company_sentiment_data():
  if company_sentiment_list is None:
        print("No data in the company sentiment list.")
        return
  for company in company_sentiment_list:
    print(company)

In [ ]:
print_company_sentiment_data()

CompanySentiment(url='https://www.cnn.com/2023/05/16/business/elon-musk-george-soros/index.html', name='Tesla', sentiment='negative')
CompanySentiment(url='https://www.cnn.com/2023/05/16/business/elon-musk-tesla-hiring-memo/index.html', name='Tesla', sentiment='negative')
CompanySentiment(url='https://www.cnn.com/2023/05/15/business/elon-musk-jeffrey-epstein-virgin-islands-subpoena/index.html', name='Tesla', sentiment='negative')
CompanySentiment(url='https://www.cnn.com/2023/05/12/investing/premarket-stocks-trading/index.html', name='Tesla', sentiment='negative')
CompanySentiment(url='https://www.cnn.com/2023/05/12/economy/china-tesla-recall-intl-hnk/index.html', name='Tesla', sentiment='negative')
CompanySentiment(url='https://www.cnn.com/cnn-underscored/deals/best-online-sales-right-now-2023-05-16', name='Apple', sentiment='positive')
CompanySentiment(url='https://www.cnn.com/2023/05/16/investing/berkshire-hathaway-taiwan-tsmc-stock-exit-hnk-intl/index.html', name='Apple', sentiment